<a href="https://colab.research.google.com/github/shiviagarwal21/ml-devops/blob/master/mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.datasets import mnist
from keras.utils import np_utils
import keras

# loads the MNIST dataset
(x_train, y_train), (x_test, y_test)  = mnist.load_data()

# Lets store the number of rows and columns
img_rows = x_train[0].shape[0]
img_cols = x_train[1].shape[0]

# Getting our date in the right 'shape' needed for Keras
# We need to add a 4th dimenion to our date thereby changing our
# Our original image shape of (60000,28,28) to (60000,28,28,1)
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

# store the shape of a single image 
input_shape = (img_rows, img_cols, 1)

# change our image type to float32 data type
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize our data by changing the range from (0 to 255) to (0 to 1)
x_train /= 255
x_test /= 255

# Now we one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

num_classes = y_test.shape[1]
num_pixels = x_train.shape[1] * x_train.shape[2]
accuracy=0
conv=1
cnt=0
epochs=2
while True:
    cnt+=1
    # create model
    model = Sequential()

    #sets of CRP (Convolution, RELU, Pooling)
    for x in range(conv):
        model.add(Conv2D(20, (5, 5),
                        padding = "same", 
                        input_shape = input_shape))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))

    # Fully connected layers (w/ RELU)
    model.add(Flatten())
    model.add(Dense(500))
    model.add(Activation("relu"))

    # Softmax for classification
    model.add(Dense(num_classes))
    model.add(Activation("softmax"))
            
    model.compile(loss = 'categorical_crossentropy',
                optimizer = keras.optimizers.Adadelta(),
                metrics = ['accuracy'])
        
    print(model.summary())

    # Training Parameters
    batch_size = 128

    history = model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_test, y_test),
            shuffle=True)

    model.save("mnist_LeNet.h5")

    # Evaluate the performance of our trained model
    scores = model.evaluate(x_test, y_test, verbose=1)
    print('Test loss = ', scores[0])
    print('Test accuracy = ', scores[1])
    accuracy=scores[1]
    print(accuracy)
    if accuracy < 0.985 and cnt<2:
        conv+=1
        continue
    elif accuracy < 0.985 :
        epochs+=1
    else:
        break 

file = open("accuracy.txt","w")
file.write(str(scores[1]*100))
file.close()

model.save("mnist_LeNet_correct.h5")

Using TensorFlow backend.


11493376/11490434 [==============================] - 0s 0us/step
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 20)        520       
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 20)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 20)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3920)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               1960500   
_________________________________________________________________
activation_2 (Activation)    (None, 500)               0         
_______________________________________________________